In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable
from datetime import datetime

In [0]:
df_hospedes = (
    spark.read.format("delta").load("/Volumes/workspace/default/raw/tb_hospedes")
    .select(
        F.col("hospede_id"),
        F.col("nome"),
        F.col("data_nascimento"),
        F.col("sexo"),
        F.col("nacionalidade"),
        F.col("tipo_cliente"),
        F.col("email"),
        F.col("telefone")
    )
    .withColumn("data_nascimento", F.to_date(F.col("data_nascimento")))
    .withColumn("idade", (F.datediff(F.current_date(), F.col("data_nascimento")) / 365).cast("integer"))
    .withColumn("start_date", F.current_date())
    .withColumn("update_date", F.lit(None).cast("date"))
)

display(df_hospedes.limit(5))

In [0]:
catalog_path = "/Volumes/workspace/default/trusted/tb_hospedes"

if DeltaTable.isDeltaTable(spark, catalog_path):
    delta_table = DeltaTable.forPath(spark, catalog_path)
    
    # Merge (upsert) com atualização da update_date
    delta_table.alias("target").merge(
        df_hospedes.alias("source"),
        "target.hospede_id = source.hospede_id"  # chave única
    ).whenMatchedUpdate(
        set = {
            # Atualiza todas as colunas exceto update_date
            **{col: f"source.{col}" for col in df_hospedes.columns if col != "update_date"},
            # Atualiza a update_date com a data atual
            "update_date": "current_date()"
        }
    ).whenNotMatchedInsertAll() \
     .execute()
else:
    # Se não existir, cria a tabela Delta incluindo a update_date
    df_hospedes.write.format("delta").save(catalog_path)